In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from utils import *
from operator import itemgetter
import json

/tmp/ipykernel_4463/4039375063.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-05 16:46:29.206604: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-05 16:46:29.812846: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-05 16:46:29.813003: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin 

In [2]:
top10_df = pd.read_csv("./data/content_top10_df.csv")

top10_df

,movie id,num ratings,ave rating,title,genres
0,4993,198,4.106061,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
1,5952,188,4.021277,"Lord of the Rings: The Two Towers, The",Adventure|Fantasy
2,7153,185,4.118919,"Lord of the Rings: The Return of the King, The",Action|Adventure|Drama|Fantasy
3,4306,170,3.867647,Shrek,Adventure|Animation|Children|Comedy|Fantasy|Ro...
4,58559,149,4.238255,"Dark Knight, The",Action|Crime|Drama
5,6539,149,3.778523,Pirates of the Caribbean: The Curse of the Bla...,Action|Adventure|Comedy|Fantasy
6,79132,143,4.066434,Inception,Action|Crime|Drama|Mystery|Sci-Fi|Thriller
7,6377,141,3.960993,Finding Nemo,Adventure|Animation|Children|Comedy
8,4886,132,3.871212,"Monsters, Inc.",Adventure|Animation|Children|Comedy|Fantasy
9,7361,131,4.160305,Eternal Sunshine of the Spotless Mind,Drama|Romance|Sci-Fi


In [3]:
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3 # Removes user id, rating couting and avg rating
num_item_features = item_train.shape[1] - 1 # Remove movie id
uvs = 3 # user genre vector start
ivs = 3 # item genre vector start
u_s = 3 # user start of features columns to use in training
i_s = 1 # items start of features columns to use in training

In [4]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9


In [5]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[movie id],year,ave rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
6874,2003,4.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,1,0,1,0,0,0,0,0,1
46970,2006,3.2,1,0,0,0,1,0,0,0,0,0,0,0,0,0
48516,2006,4.3,0,0,0,0,0,1,0,1,0,0,0,0,0,1
58559,2008,4.2,1,0,0,0,0,1,0,1,0,0,0,0,0,0


In [6]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [4.  3.5 4.  4.  4.5]


In [7]:
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))

In [8]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test = train_test_split(y_train, train_size = 0.80, shuffle=True, random_state=1)

In [9]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
1,0,-1.0,-0.8,-0.7,0.1,-0.0,-1.2,-0.4,0.6,-0.5,-0.5,-0.1,-0.6,-0.6,-0.7,-0.7
0,1,-0.7,-0.5,-0.7,-0.1,-0.2,-0.6,-0.2,0.7,-0.5,-0.8,0.1,-0.0,-0.6,-0.5,-0.4
-1,-1,-0.2,0.3,-0.4,0.4,0.5,1.0,0.6,-1.2,-0.3,-0.6,-2.3,-0.1,0.0,0.4,-0.0
0,-1,0.6,0.5,0.5,0.2,0.6,-0.1,0.5,-1.2,0.9,1.2,-2.3,-0.1,0.0,0.2,0.3
-1,0,0.7,0.6,0.5,0.3,0.5,0.4,0.6,1.0,0.6,0.3,0.8,0.8,0.4,0.7,0.7


In [10]:
num_outputs = 32

user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear')
])

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear')
])

input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

output = tf.keras.layers.Dot(axes=1)([vu, vm])

model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 14)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 sequential (Sequential)     (None, 32)                   147232    ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 32)                   148256    ['input_2[0][0]']             
                                                                                              

In [11]:
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss=cost_fn)

In [12]:
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=10)

Epoch 1/10
1273/1273 [==============================] - 10s 7ms/step - loss: 0.1247
Epoch 2/10
1273/1273 [==============================] - 7s 6ms/step - loss: 0.1169
Epoch 3/10
1273/1273 [==============================] - 8s 6ms/step - loss: 0.1132
Epoch 4/10
1273/1273 [==============================] - 7s 6ms/step - loss: 0.1106
Epoch 5/10
1273/1273 [==============================] - 7s 6ms/step - loss: 0.1076
Epoch 6/10
1273/1273 [==============================] - 8s 6ms/step - loss: 0.1044
Epoch 7/10
1273/1273 [==============================] - 6s 4ms/step - loss: 0.1022
Epoch 8/10
1273/1273 [==============================] - 5s 4ms/step - loss: 0.1003
Epoch 9/10
1273/1273 [==============================] - 5s 4ms/step - loss: 0.0984
Epoch 10/10
1273/1273 [==============================] - 5s 4ms/step - loss: 0.0958


In [13]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 [==============================] - 1s 1ms/step - loss: 0.0963


0.09633439779281616

In [14]:
def sq_distance(a, b):
    return np.sum((a - b) ** 2)

In [15]:
def compute_distance_matrix(vms):
    m = vms.shape[0]
    M = np.zeros((m, m))

    for i in range(m):
        for j in range(m):
            M[i][j] = sq_distance(vms[i, :], vms[j, :])

    return M

In [16]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))
vm_m = item_NN(input_item_m)
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)
model_m = tf.keras.Model(input_item_m, vm_m)
model_m.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 16)]              0         
                                                                 
 sequential_1 (Sequential)   (None, 32)                148256    
                                                                 
 tf.math.l2_normalize_2 (TF  (None, 32)                0         
 OpLambda)                                                       
                                                                 
Total params: 148256 (579.12 KB)
Trainable params: 148256 (579.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])

27/27 [==============================] - 0s 1ms/step


In [18]:
distance_matrix = compute_distance_matrix(vms)

In [19]:
m = distance_matrix.shape[0]
similar_movies_list = []

distance_matrix = ma.masked_array(distance_matrix, mask=np.identity(distance_matrix.shape[0]))

for i in range(m):
    movie = dict()
    min_idx = np.argmin(distance_matrix[i, i:])
    movie1_id = int(item_vecs[i, 0])
    movie2_id = int(item_vecs[min_idx, 0])
    distance = distance_matrix[i][min_idx]
    movie["movie1_id"] = movie1_id
    movie["movie2_id"] = movie2_id
    movie["distance"] = distance 
    
    similar_movies_list.append(movie)

In [20]:
top_10_similar_movies = sorted(similar_movies_list, key=itemgetter('distance'))[0:9]

In [21]:
disp = [["movie1", "movie1_id", "genres", "movie2", "movie2_id", "genres", "distance"]]

for i in range(len(top_10_similar_movies)):
    movie1_id = top_10_similar_movies[i]["movie1_id"]
    movie2_id = top_10_similar_movies[i]["movie2_id"]
    distance = top_10_similar_movies[i]["distance"]
    disp.append([
        movie_dict[movie1_id]["title"], movie1_id, movie_dict[movie1_id]["genres"],
        movie_dict[movie2_id]["title"], movie2_id, movie_dict[movie2_id]["genres"],
        distance
    ])

table = tabulate.tabulate(disp, tablefmt="html", headers="firstrow")
table

movie1,movie1_id,genres,movie2,movie2_id,genres,distance
Save the Last Dance (2001),4054,Drama|Romance,Mona Lisa Smile (2003),7154,Drama|Romance,0.00144842
"Downfall (Untergang, Der) (2004)",31410,Drama,Blow (2001),4239,Crime|Drama,0.00290166
Knocked Up (2007),52973,Comedy|Drama|Romance,Punch-Drunk Love (2002),5673,Comedy|Drama|Romance,0.00315198
Saved! (2004),8366,Comedy|Drama,Million Dollar Baby (2004),30707,Drama,0.00986056
"Perks of Being a Wallflower, The (2012)",96821,Drama|Romance,Far from Heaven (2002),5812,Drama|Romance,0.0118723
"Bucket List, The (2007)",56587,Comedy|Drama,Death to Smoochy (2002),5265,Comedy|Crime|Drama,0.019001
Thank You for Smoking (2006),44195,Comedy|Drama,Hotel Rwanda (2004),30749,Drama,0.021933
Blow (2001),4239,Crime|Drama,Far from Heaven (2002),5812,Drama|Romance,0.0270825
Ray (2004),8958,Drama,"Hours, The (2002)",5992,Drama|Romance,0.027391


In [22]:
recommendations = dict()

recommendations["vector"] = item_vecs[:, 0].astype(int).tolist()
recommendations["matrix"] = distance_matrix.tolist()

with open("./output/recommendations.json", "w", encoding="utf-8") as f:
    json.dump(recommendations, f, ensure_ascii=True, indent=4)

In [30]:
movie_a_id = 4054 # Save the Last Dance (2001)
movie_b_id = 7154 # Mona Lisa Smile (2003)	
movie_a_title = movie_dict[movie_a_id]["title"]
movie_b_title = movie_dict[movie_b_id]["title"]
movie_a_matrix_idx = recommendations["vector"].index(movie_a_id)
movie_b_matrix_idx = recommendations["vector"].index(movie_b_id)
movies_a_b_distance = recommendations["matrix"][movie_a_matrix_idx][movie_b_matrix_idx]

print(f"Distance between \"{movie_a_title}\" (movie A) and \"{movie_b_title}\" (movie B) is {movies_a_b_distance}")

Distance between "Save the Last Dance (2001)" (movie A) and "Mona Lisa Smile (2003)" (movie B) is 0.001448422553949058
